In [1]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('/content/preprocessedData.csv')


In [3]:
from sklearn.model_selection import train_test_split
target = np.array(df['HITS'])
df.drop(labels='HITS',axis=1,inplace=True)
features = np.array(df)
train_features, test_features, train_labels, test_labels = train_test_split(features, target, test_size = 0.25, random_state = 42)

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (446802, 68)
Training Labels Shape: (446802,)
Testing Features Shape: (148935, 68)
Testing Labels Shape: (148935,)


In [4]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = train_features.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               8832      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 239,489
Trainable params: 239,489
Non-trainable params: 0
__________________________________________________

In [5]:
NN_model.fit(train_features, train_labels, epochs=50, batch_size=32, validation_split = 0.2)

Epoch 1/50
11171/11171 [==============================] - 37s 3ms/step - loss: 1.2498 - mean_absolute_error: 1.2498 - val_loss: 0.2872 - val_mean_absolute_error: 0.2872
Epoch 2/50
11171/11171 [==============================] - 36s 3ms/step - loss: 0.2859 - mean_absolute_error: 0.2859 - val_loss: 0.2873 - val_mean_absolute_error: 0.2873
Epoch 3/50
11171/11171 [==============================] - 36s 3ms/step - loss: 0.2868 - mean_absolute_error: 0.2868 - val_loss: 0.2872 - val_mean_absolute_error: 0.2872
Epoch 4/50
11171/11171 [==============================] - 36s 3ms/step - loss: 0.2845 - mean_absolute_error: 0.2845 - val_loss: 0.2871 - val_mean_absolute_error: 0.2871
Epoch 5/50
11171/11171 [==============================] - 36s 3ms/step - loss: 0.2845 - mean_absolute_error: 0.2845 - val_loss: 0.2873 - val_mean_absolute_error: 0.2873
Epoch 6/50
11171/11171 [==============================] - 35s 3ms/step - loss: 0.2850 - mean_absolute_error: 0.2850 - val_loss: 0.2870 - val_mean_absolute_

In [8]:
model_json = NN_model.to_json()
with open("/content/nn_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
NN_model.save_weights("/content/nn_model.h5")
print("Saved model to disk")

Saved model to disk


In [10]:
preds = NN_model.predict(test_features)
from sklearn.metrics import mean_squared_error
mean_squared_error(test_labels, preds)

0.9460722749268206